In [ ]:
library(Seurat)
library(Matrix)
library(tidyverse)

source("../../util/sc_preprocess.R")

In [ ]:
# The following RDS files are products from import_DGE.R
dge_base   <- readRDS("vFeb2021_Drop_original_rawdge.rds")
dge_roundtrip <- readRDS("Drop_roundtrip.drop.rds")

In [ ]:
bclist_fwd <- read.table("barcode_correspondence_data//FigS3/Drop/fwd/merge_correspondence.tsv",header=F)
bclist_rev <- read.table("barcode_correspondence_data//FigS3/Drop/rev/merge_correspondence.tsv",header=F)

In [ ]:
bclist <- merge(bclist_fwd,bclist_rev,by.x=2,by.y=1)

In [ ]:
colnames(bclist) <- c("sim","orig","roundtrip")

In [ ]:
dim(bclist)

In [ ]:
# colnames(dge_base) <- gsub("-1","",colnames(dge_base))
# colnames(dge_roundtrip) <- gsub("-1","",colnames(dge_roundtrip))
dge_new <- refresh_dge(dge_roundtrip,bclist$orig,bclist$roundtrip)

In [ ]:
dim(dge_new)

In [ ]:
dge_target_list <- list(dge_new)
use_cells <- get_cells_used(dge_base,dge_target_list,percent_mito = 20)

dge_base_intersect   <- dge_base[,use_cells]
dge_new_intersect  <- dge_new[,use_cells]

In [ ]:
# intersection
ncol(dge_base_intersect)
ncol(dge_new_intersect)

In [ ]:
orig.intersect.seu   <- CreateSeuratObject(dge_base_intersect)
roundtrip.seu    <- CreateSeuratObject(dge_new_intersect)

In [ ]:
high_var_genes <- getVarGenes_untilPCA(orig.intersect.seu,num_varFeatures = 5000)

In [ ]:
orig.intersect.seu   <- getVarGenes_untilPCA(orig.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)
roundtrip.seu    <- getVarGenes_untilPCA(roundtrip.seu,var_genes=high_var_genes,num_varFeatures = 5000)

In [ ]:
orig.intersect.seu   <- seurat_processing_UMAP(orig.intersect.seu,dimlimit = 20,res = 0.6)
roundtrip.seu  <- seurat_processing_UMAP(roundtrip.seu,dimlimit = 20,res = 0.6)

In [ ]:
store_dir <- "intermediate_data/"

saveRDS(orig.intersect.seu, paste0(store_dir,"Aug2022_processed_Drop_original_Seurat.intersectionWithRndTrip.rds"))
saveRDS(roundtrip.seu, paste0(store_dir,"Aug2022_processed_Drop_roundTrip_Seurat.rds"))

#### Use cell state labels identified in the original seurat obj used in Figure 4

In [ ]:
orig.fig4.seu <- readRDS("Drop_original_Seurat.rds")

In [ ]:
celltype_v_fig4 <- orig.fig4.seu$seurat_clusters
celltype_v_fig4 <- celltype_v_fig4[colnames(orig.intersect.seu)]

In [ ]:
head(celltype_v_fig4)

In [ ]:
colnames(orig.intersect.seu) %>% head
colnames(roundtrip.seu) %>% head

In [ ]:
#Fig drawing
draw_umap(seu_base = orig.intersect.seu,
          seu_to10x = orig.intersect.seu,
          seu_todrop = roundtrip.seu,
          col_seed = 22,
          w=6,
          h=6,
          is_pdf=F,
          celltype_v=celltype_v_fig4,
          outdir = "figure_out/",
          outname = "Aug2022_Drop_roundtrip")

In [ ]:
#Export CellCellDist
v.vln.todrop <- get_dist_pcaspace_scatter(orig.intersect.seu,
                                          roundtrip.seu,
                                          outdir = "figure_out/",
                                          outname = "Aug2022_Drop_roundtrip")